In [1]:
import os
import glob
import shutil
import cv2
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import json_tricks as json
import mmcv
import mmengine
import numpy as np
import pandas as pd

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']
cm = {"cather":4.72, "daniel":4.74, "dominic":3.9, "jack":3.91, "jakaria":4.20, "jimmy":4.60, "vicky":4.69, "wayne":4.75,
      "gaby":4.05, "matt":5.0, "toby":4.26, "wendy":4.14, "willy":4.51}
ear_types=["free","attached"]
acupoints_name = ["HX6(肛門)", "HX3(尿道)", "HX1(耳中)", "TF4(神門)", "AH9(腰柢椎)", "AH11(胸椎)", "CO5(十二指腸)", 
                  "CO4(胃)", "CO3(賁門)", "CO14(肺)", "CO15(心)", "CO18(內分泌)", "HX2(輪2)", "AT2(緣中)", 
                  "LO8(扁桃體)", "LO1(牙)" , "LO2(舌)", "LO3(頷)", "LO4(垂前)", "LO5(眼)", "LO6(內耳)"]

In [3]:
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
register_all_modules()

In [4]:
def occlude_within_bbox(mask_size):
    kpt_folder = "../keypoint"
    for ear_type in ear_types:
        names = os.listdir(os.path.join(kpt_folder, ear_type, "5_mmdet_bboxes_json"))
        for name in names:
            for deg in degrees:
                bboxes_json = glob.glob(os.path.join(kpt_folder, ear_type, "5_mmdet_bboxes_json", name, deg, "*.json"))[0]
                
                with open(bboxes_json,"r") as bboxes_file:
                    bboxes = json.load(bboxes_file)

                interval = len(bboxes) / 3
                indexes = [0, int(interval), int(interval*2), len(bboxes)-1]
                
                for index in indexes:
                    img = cv2.imread(os.path.join(kpt_folder, ear_type, "4_return_to_ori_size", name, deg, "{frame}.png".format(frame = index)))
                    x, y, w, h = bboxes[str(index)]
                    
                    img_path = os.path.join(kpt_folder, ear_type, "9_occlusion_img", name, deg, index)
                    if not os.path.exists(img_path):
                        os.makedirs(img_path)
                    
                    count = 0
                    for i in range(0, w, mask_size[0]):
                        for j in range(0, h, mask_size[1]):
                            img_copy = img.copy()
                            img_copy[y+j:y+j+int(mask_size[1]), x+i:x+i+int(mask_size[0])] = (0, 0, 0)
                            cv2.imwrite(os.path.join(img_path, "{x}.png".format(x = count)), img_copy)
                            cv2.imshow("Modified Image", img_copy)
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()
                            count += 1
                

../keypoint\free\5_mmdet_bboxes_json\cather\15cm_0mm_0deg\*.json


In [3]:
try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False


def process_one_image(img,
                      detector,
                      pose_estimator):
    
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate((pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                   pred_instance.scores > 0.3)]
    bboxes = bboxes[nms(bboxes, 0.3), :4]


    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results)

    return data_samples.get('pred_instances', None)


def main():

    rtmdet_cfg = '../configs/rtmdet/rtmdet_nano_320-8xb32_coco-ear.py'
    rtmdet_ckp = '../work_dirs/rtmdet_nano_320-8xb32_coco-ear/epoch_120.pth'

    

    kpt_folder = "../keypoint"
    cfg_folder ="../mmpose/configs/body_2d_keypoint/rtmpose/coco"
    rtmdet_model = init_detector(
        rtmdet_cfg, 
        rtmdet_ckp, 
        device="cuda:0")
    rtmdet_model-.cfg = adapt_mmdet_pipeline(rtmdet_model.cfg)

    for ear_type in ear_types:
        
        names = os.listdir(os.path(kpt_folder, ear_type))


        for name in names:
             
            rtmpose_cfg = os.path.join(cfg_folder,"rtmpose-s_8xb256-420e_coco-256x192_custom_{et}.py".format(et = ear_type))
            rtmpose_ckp = glob.glob(os.path.join(kpt_folder, ear_type, "7_result", name,"best_coco*.pth"))[0]
            
            pose_estimator = init_pose_estimator(
            rtmpose_cfg
            rtmpose_ckp,
            device="cuda:0",
            cfg_options=dict(
                model=dict(test_cfg=dict(output_heatmaps=False))))

            for deg in degrees:
                indexes = os.list(os.path.join(kpt_folder, ear_type, "9_occlusion_img", name, deg))
                
                for index in indexes:
                    imgs = os.list(os.path.join(kpt_folder, ear_type, "9_occlusion_img", name, deg, index))
                    imgs = sorted(imgs, key = lambda s : int(os.path.splitext(os.path.basename(s))[0]), reverse = False)
                    df = pd.DataFrame()
                    for img in imgs:
                        image = cv2.imread(os.path.join(kpt_folder, ear_type, "9_occlusion_img", name, deg, index, img))
                        pred_instances = process_one_image(image, detector, pose_estimator)
                        kpts = pred_instances_list[0]["keypoints"]
                        kpts_pd = {}
                        for i in range(0, len(kpts)):
                            x, y  = kpts[i]
                            kpts_pd[i] = str(x)+str(",")+str(y)
                        ser = pd.DataFrame(data=kpts_pd, index = [total_count])
                        df = pd.concat([df, ser])
                    occlusion_pred = os.path.join(kpt_folder, ear_type, "9_occlusion_pred", name, deg, index)
                    
                    if not os.path.isdir(occlusion_pred):
                        os.makedirs(occlusion_pred_json)
                    df.to_csv(os.path.join(occlusion_pred, "pred.csv"))



if __name__ == '__main__':
    occlude_within_bbox((50,50))
    main()

Loads checkpoint by local backend from path: ../mmdetection/work_dirs/rtmdet_nano_320-8xb32_coco-ear/epoch_120.pth
Loads checkpoint by local backend from path: ../keypoint/attached/7_result\gaby\best_coco_AP_epoch_70.pth
Loads checkpoint by local backend from path: ../keypoint/attached/7_result\matt\best_coco_AP_epoch_40.pth
Loads checkpoint by local backend from path: ../keypoint/attached/7_result\toby\best_coco_AP_epoch_10.pth
Loads checkpoint by local backend from path: ../keypoint/attached/7_result\wendy\best_coco_AP_epoch_50.pth
Loads checkpoint by local backend from path: ../keypoint/attached/7_result\willy\best_coco_AP_epoch_20.pth
